# MiniRun5 LRS output tutorial

In [ ]:
#!git clone https://github.com/peter-madigan/h5flow
!pip install ./h5flow

In [ ]:
#!git clone https://github.com/DUNE/ndlar_flow
!pip install ./ndlar_flow

In [ ]:
!wget https://portal.nersc.gov/project/dune/data/2x2/simulation/tutorial_jan2024_inputs/flow_output_example_MR5beta_LROsteroids.proto_nd_flow.h5

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import matplotlib.patches as mpatches
import itertools

from h5flow.data import dereference
from proto_nd_flow.util.lut import LUT


In [ ]:
def h5_tree(val, pre='',skip_ref=False):
    items = len(val)
    for key, val in val.items():
        items -= 1
        if items == 0:
            # the last item
            if type(val) == h5py._hl.group.Group:
                print(pre + '└── ' + key)
                if key=="ref" and skip_ref:
                    continue
                h5_tree(val, pre+'    ')
            else:
                print(pre + '└── ' + key + ' (%d)' % len(val))
        else:
            if type(val) == h5py._hl.group.Group:
                print(pre + '├── ' + key)
                h5_tree(val, pre+'│   ')
            else:
                print(pre + '├── ' + key + ' (%d)' % len(val))

## Import ndlar_flow file

In [ ]:
#f = h5py.File("flow_output_example_MR5beta_LROsteroids.proto_nd_flow.h5")
f = h5py.File("../MiniRun5_1E19_RHC.larnd.00666.LARNDSIM.noisefix.proto_nd_flow.hdf5")
h5_tree(f,skip_ref=True)

In [ ]:
h5_tree(f["light"])

In [ ]:
h5_tree(f["geometry_info"])

## The More Basic Structure Check:

In [ ]:
my_func = lambda name,dset : print(name) if isinstance(dset, h5py.Dataset) \
    else None
f.visititems(my_func)

### When Referencing: 

There are two types of reference branch: 
1. a.../ref/b.../ref_region
2. a.../ref/b.../ref

Typically, you use both to reference (unless you use the h5flow dereference function)  
Suppose you want to find the calib_final_hits associated with a given event ID.    
charge/event/data and charge/calib_final_hits/data are different sizes! 

How do we start with charge/events/data and end with an appropriate index cut for calib_final_hits?

In [ ]:
ev_id = 0
event = f['charge/events/data'][ev_id]

First, we use a.../ref/b.../ref_region.   
a.../ref/b.../ref_region is an array of arrays like [[0 2][3 7]...]   
It has as many entries as a..., such that its shape is: [len(a...), 2] with keys ['start', 'stop']    
'start' is the first index in b... asociated with the ev_id.    
'stop' is the last index in b... assiciated with ev_id   
b... should be the branch you want to perform a cut on.

In [ ]:
href_start = f['charge/events/ref/charge/calib_final_hits/ref_region'][ev_id,'start']
href_stop = f['charge/events/ref/charge/calib_final_hits/ref_region'][ev_id,'stop']

Here we apply the cut to  a.../ref/b.../ref.    
This outputs an array with [ev_id, b... index] for every b... indice linked to ev_id.

In [ ]:
hit_ref = f['charge/events/ref/charge/calib_final_hits/ref'][href_start:href_stop]

Cuts need to be a sequential array, so we make sure there's only one ev_id, and the indices are ordered.   
Then, we apply the cut. 

In [ ]:
hit_ref = np.sort(hit_ref[hit_ref[:,0] == ev_id, 1])
hits_trk = f['charge/calib_final_hits/data'][hit_ref]
print(f['charge/calib_final_hits/data'].dtype.names)

If we format this as a function:

In [ ]:
def one_to_many(event_id, h5file, one_entry_field, many_entry_field):
    print(one_entry_field+'/data')
    one_index = h5file[one_entry_field+'/data'][event_id]
    print(one_entry_field+'/ref/'+many_entry_field+'/ref_region')
    href_start = h5file[one_entry_field+'/ref/'+many_entry_field+'/ref_region'][event_id, 'start']
    href_stop = h5file[one_entry_field+'/ref/'+many_entry_field+'/ref_region'][event_id, 'stop']
    h_ref = h5file[one_entry_field+'/ref/'+many_entry_field+'/ref'][href_start:href_stop]
    h_ref = np.sort(h_ref[h_ref[:,0] == event_id, 1])
    print(many_entry_field+'/data')
    h_trk = h5file[many_entry_field+'/data'][h_ref]
    
    return h_trk

Let's try it in a plot:

In [ ]:
def ndflow_charge_plot(event_id, h5file, one_entry_field, many_entry_field):
    
    hits_trk = one_to_many(event_id, h5file, one_entry_field, many_entry_field)
    
    x_pos = hits_trk['x'] # [cm], drift direction
    io_groups = hits_trk['io_group']
    y_pos = hits_trk['y']
    z_pos = hits_trk['z'] 
    
    fig,ax = plt.subplots(1,1,sharey=True, tight_layout=True, figsize=(13, 7))
    tpc_rectL = plt.Rectangle((-64.5,-65), 64, 130, linewidth=0.75, edgecolor='b', facecolor=cmap(0),zorder=-1)
    tpc_rectR = plt.Rectangle((0.5,-65), 64, 130, linewidth=0.75, edgecolor='b', facecolor=cmap(0),zorder=-1)

    ax.add_patch(tpc_rectR)
    ax.add_patch(tpc_rectL)
    
    ax.scatter(z_pos[io_groups==3],y_pos[io_groups==3],c='aqua',alpha=1,lw=1.5,s=8)
    ax.scatter(z_pos[io_groups==1],y_pos[io_groups==1],c='aqua',alpha=1,lw=1.5,s=8)
    ax.scatter(z_pos[io_groups==4],y_pos[io_groups==4],c='lightgreen',alpha=1,lw=1.5,s=8) 
    ax.scatter(z_pos[io_groups==2],y_pos[io_groups==2],c='lightgreen',alpha=1,lw=1.5,s=8)  
    ax.scatter(z_pos[io_groups==7],y_pos[io_groups==7],c='yellow',alpha=1,lw=1.5,s=8)
    ax.scatter(z_pos[io_groups==5],y_pos[io_groups==5],c='yellow',alpha=1,lw=1.5,s=8)
    ax.scatter(z_pos[io_groups==8],y_pos[io_groups==8],c='orangered',alpha=1,lw=1.5,s=8)                       
    ax.scatter(z_pos[io_groups==6],y_pos[io_groups==6],c='orangered',alpha=1,lw=1.5,s=8)
            
    ax.set_aspect("equal")
    ax.set_title('All Tracks')

    ax.set_xlabel("z [cm]")
    ax.set_ylabel("y [cm]")
    b_patch = mpatches.Patch(color='aqua', label='io_groups: 1,3')
    g_patch = mpatches.Patch(color='lightgreen', label='io_groups: 2,4')
    y_patch = mpatches.Patch(color='yellow', label='io_groups: 5,7')
    o_patch = mpatches.Patch(color='orangered', label='io_groups: 6,8')
    ax.legend(handles=[b_patch, g_patch,y_patch,o_patch],
          loc='lower left',ncol = 4, bbox_to_anchor=(-0.11,-0.2))
    plt.show()

In [ ]:
cmap = cm.jet
ndflow_charge_plot(5, f, 'charge/events', 'charge/calib_final_hits')

Now with light:

Note, for event 5, we're mainly in io_group 1.

In [ ]:
light_evts = one_to_many(5, f, 'charge/events', 'light/events')
print('Light/events keys:',light_evts.dtype.names)
light_wvfms = one_to_many(light_evts['id'][0], f, 'light/events', 'light/wvfm')
print('Light/wvfms key:', light_wvfms.dtype.names)
print('samples shape:', np.shape(light_wvfms['samples']))

### A note on geometry:

Every ADC has 64 available channels.   
Per ADC, the 2x2 light LRS uses 48 of those 64 channels to collect light waveforms.   
We can use f['light/events/data']['wvfm_valid'] to select for this. 

In [ ]:
valid_wvfm = light_evts['wvfm_valid']
print('Valid Waveform Shape:',np.shape(valid_wvfm))
wvfms_0 = light_wvfms['samples'][0,0,:][valid_wvfm[0,0,:]==1]
print('New Light Waveform Shape:',np.shape(wvfms_0))

In [ ]:
SAMPLES = 1000
fig = plt.figure(figsize=(12,3))
plt.plot(np.linspace(0,SAMPLES-1,SAMPLES),wvfms_0[5,:], 'm-')
plt.title('Single Light Waveform', fontsize=16)
plt.xlabel(r'Time Sample [0.01 $\mu$s]', fontsize=14)
plt.ylabel('SiPM Channel Output', fontsize=14)
#plt.legend(fontsize=14)
plt.show()

### Import geometry info

Realised yesterday that "geometry_info" in the current form is not useful. So here for the moment a bush fix...

In [ ]:
sipm_abs_pos = LUT.from_array(f["geometry_info/sipm_abs_pos"].attrs["meta"],f["geometry_info/sipm_abs_pos/data"])

# sipm_abs_pos[(adc,chan)] = [x,y,z] in cm

In [ ]:
sipm_rel_pos = LUT.from_array(f["geometry_info/sipm_rel_pos"].attrs["meta"],f["geometry_info/sipm_rel_pos/data"])

# sipm_rel_pos[(adc,chan)] = [tpc, side, vertical position]
# side: 0->left 1->right looking in drift direction
# vertical position: index starting from bottom (0-23)

In [ ]:
det_bounds = LUT.from_array(f["geometry_info/det_bounds"].attrs["meta"],f["geometry_info/det_bounds/data"])

# det_bounds[(tpc,det)] = [[x_min, y_min, z_min],[x_max, y_max, z_max]] in cm 

In [ ]:
det_rel_pos = LUT.from_array(f["geometry_info/det_rel_pos"].attrs["meta"],f["geometry_info/det_rel_pos/data"])

# det_rel_pos[(adc,chan)] = [tpc, side, vertical position]
# side: 0->left 1->right looking in drift direction
# vertical position: index starting from bottom (0-15)

In [ ]:
det_id = LUT.from_array(f["geometry_info/det_id"].attrs["meta"],f["geometry_info/det_id/data"])

# det_id[(adc,chan)] = det

## Plot waveform with hits

Here event = trigger-event =spill !

In [ ]:
lwvfm = f['light/swvfm/data']
levent = f['light/events/data']
leventhitref = f['light/events/ref']['light/sum_hits']['ref']
leventwvfmref = f['light/events/ref']['light/swvfm']['ref']
lhits = f['light/sum_hits/data']
print("WVFM shape:\t",lwvfm["samples"].shape)
print("Events shape:\t",levent.shape)
print("Hits shape:\t",lhits.shape)

In [ ]:
spill=0

In [ ]:
#Get assosciated light data
lev_id = spill #To be fixed: Only true for beam only simulation
lev_wvfm = dereference(lev_id,leventwvfmref,lwvfm)[0]["samples"]
lev_hits = dereference(lev_id,leventhitref,lhits)[0]
acl_det = np.array([0,4,8,12])

#Plot waveform
fig, ax = plt.subplots(1,2,figsize=(8,4))
for i,j in itertools.product(range(lev_wvfm[0].shape[0]),range(lev_wvfm[0].shape[1])):
    if np.isin(j,acl_det):
        ax[0].plot(np.arange(1000),lev_wvfm[0][i,j,:],linewidth=0.5)
    else:
        ax[1].plot(np.arange(1000),lev_wvfm[0][i,j,:],linewidth=0.5)
#ax[0].set_ylim(*ax[0].get_ylim()) #Stupid matplot bug
#ax[1].set_ylim(*ax[1].get_ylim()) #Stupid matplot bug
ax[0].set_ylim(-500,500) #Stupid matplot bug
ax[1].set_ylim(-500,500) #Stupid matplot bug

#Plot hits
print("#Hits found: ",(~np.isnan(lev_hits["sample_idx"])).sum())
if np.any(lev_hits["sample_idx"]):
    hit_array = np.zeros((len(lev_hits),4))
    hit_array[:,0] = lev_hits["sample_idx"].ravel()
    hit_array[:,2] = lev_hits["sample_idx"].ravel()
    hit_array[:,1] = ax[0].get_ylim()[0]
    hit_array[:,3] = ymax=ax[0].get_ylim()[0]/3
    hit_plot = ax[0].plot(hit_array[:,::2].T, hit_array[:, 1::2].T,color="red")
    hit_plot = ax[1].plot(hit_array[:,::2].T, hit_array[:, 1::2].T,color="red")
    plt.legend([hit_plot[0]],['sum hits'],loc='best')

#ax.set_title('Spill %d' %spill)
#ax.set_xlabel("sample [16ns]")
#ax.set_ylabel("ADC value")
spill+=1

In [ ]:
#Get assosciated light data
lev_id = spill #To be fixed: Only true for beam only simulation
lev_wvfm = dereference(lev_id,leventwvfmref,lwvfm)[0]["samples"]
lev_hits = dereference(lev_id,leventhitref,lhits)[0]

#Plot waveform
fig, ax = plt.subplots(1,figsize=(4,4))
for i,j in itertools.product(range(lev_wvfm[0].shape[0]),range(lev_wvfm[0].shape[1])):
    ax.plot(np.arange(1000),lev_wvfm[0][i,j,:],linewidth=0.5)
ax.set_ylim(*ax.get_ylim()) #Stupid matplot bug

#Plot hits
print("#Hits found: ",(~np.isnan(lev_hits["sample_idx"])).sum())
if np.any(lev_hits["sample_idx"]):
    hit_array = np.zeros((len(lev_hits),4))
    hit_array[:,0] = lev_hits["sample_idx"].ravel()
    hit_array[:,2] = lev_hits["sample_idx"].ravel()
    hit_array[:,1] = ax.get_ylim()[0]
    hit_array[:,3] = ymax=ax.get_ylim()[0]/3
    hit_plot = plt.plot(hit_array[:,::2].T, hit_array[:, 1::2].T,color="red")
    plt.legend([hit_plot[0]],['sum hits'],loc='best')

ax.set_title('Spill %d' %spill)
ax.set_xlabel("sample [16ns]")
ax.set_ylabel("ADC value")
spill+=1

### Hit structure
| Name              | Type             | Description                                                                                             |
| ----------------- | ---------------- | ------------------------------------------------------------------------------------------------------- |
| id                | u4,              | unique identifier                                                                                       |
| adc_id / tpc      | u1,              | tpc or adc index                                                                                        |
| chan / det        | u1,              | detector or channel index                                                                               |
| pos / boundary    | f4(3) / f4(3,2), | (x,y,z) center of sipm / ((xmin,xmax), (ymin,ymax), (zmin,zmax)) boundary of detector sensitive surface |
| sample_idx        | u2,              | sample index of peak within waveform                                                                    |
| ns                | f8,              | WR timestamp of waveform [ns]                                                                           |
| busy_ns           | f8,              | timestamp of peak relative to trigger [ns]                                                              |
| samples           | f4(2\*near+1,),  | sample value around peak (±near_samples)                                                                |
| sum               | f4,              | sum of sample values (out to ±near_samples)                                                             |
| max               | f4,              | peak value                                                                                              |
| sum_spline        | f4,              | integral of spline around peak (out to ±near_samples)                                                   |
| max_spline        | f4,              | maximum of spline around peak                                                                           |
| ns_spline         | f4,              | offset from center sample for maximum of spline [ns]                                                    |
| rising_spline     | f4,              | projection of spline to rising edge zero-crossing (offset from center sample) [ns]                      |
| rising_err_spline | f4,              | an estimate of the error on the rising edge zero-                                                       |
| fwhm_spline       | f4,              | spline FWHM [ns] 

# Charge Light combined plot

In [ ]:
light_mode = "wvfm_sum"
#light_mode = "sum_hit"
#light_mode = "sipm_hit"

In [ ]:
levent = f['light/events/data']
if light_mode == "wvfm_sum":
    lwvfm = f['light/wvfm/data']
    leventwvfmref = f['light/events/ref']['light/wvfm']['ref']
elif light_mode == "sum_hit":
    lhits = f['light/sum_hits/data']
    leventhitref = f['light/events/ref']['light/sum_hits']['ref']
else:
    lhits = f['light/sipm_hits/data']
    leventhitref = f['light/events/ref']['light/sipm_hits']['ref']


chits = f['/charge/calib_prompt_hits/data']
cevents = f['charge/events/data']
ceventchitref = f['charge/events/ref/charge/calib_prompt_hits']['ref']
ceventleventref = f['charge/events/ref/light/events']['ref']

In [ ]:
spill = 45

In [ ]:
cevents.dtype

In [ ]:
### Event display

#Get light data
lev_id = spill #To be fixed: Only true for beam only simulation
lev_wvfm = dereference(lev_id,leventwvfmref,lwvfm)[0]["samples"]
lev_hits = dereference(lev_id,leventhitref,lhits)[0]

#Get charge data
cevent = dereference(lev_id,ceventleventref,cevents,ref_direction = (1,0),indices_only = True)[0]

print("Light utime:", levent[spill]["utime_ms"])
print("#Light hits found:", lev_hits.shape[0])
if np.any(cevent):
    cev_hits = dereference(cevent,ceventchitref,chits)
    print("Charge utime:", cevents[cevent]["unix_ts"]*1e7+cevents[cevent]["ts_start"])
else:
    print("No charge event found")
    cev_hits = np.empty(0)
    
#Prepare plot
fig = plt.figure(layout='constrained', figsize=(8, 8))
fig.suptitle("Spill %d" % spill,fontsize='xx-large')
subfigs = fig.subfigures(2, 2, wspace=0.07)
ax=[]

#Prepare color map for light
cmap=cm.viridis
if light_mode == "wvfm_sum":
    norm = colors.LogNorm(1,lev_wvfm[0].sum(axis=-1).max())
    print(lev_wvfm[0].sum(axis=-1).max())
    c = norm(lev_wvfm[0].sum(axis=-1))
else:
     norm =   plt.Normalize(0,lev_hits["max"].max())

#Define TPC borders
drift_length = 30.27225 # cm
drift_length = 35.27225 # cm
xbound=np.array([[0,drift_length],[-drift_length,0]])
ybound=np.array([-65.0,65.0])
zbound=np.array([-32.5,32.5])
mod_offsets = [
    [33.5, 0., 33.5],
    [33.5, 0., -33.5],
    [-33.5, 0., 33.5],
    [-33.5, 0., -33.5]]

for mod in range(4):
    subfigs[mod//2,mod%2].suptitle("Mod-%d"%mod)
    ax.append(subfigs[mod//2,mod%2].subplots(1,2))
    for itpc in range(2):
        ax[mod][itpc].set_xlim(*(mod_offsets[mod][2]+zbound))
        ax[mod][itpc].set_ylim(*(mod_offsets[mod][1]+ybound))
        ax[mod][itpc].set_title("TPC %d"%(mod*2+itpc))
        
        #Plot charge
        if cev_hits.shape[0]:
            c_xmask= (cev_hits["io_group"]-1==(mod*2+itpc))
            ax[mod][itpc].scatter(cev_hits[c_xmask]['z'],cev_hits[c_xmask]['y'],c=cev_hits[c_xmask]['Q'],s=0.5,cmap='viridis',norm=colors.LogNorm())
        
        #Plot light
        if light_mode == "wvfm_sum":
            for i,j in itertools.product(range(lev_wvfm[0].shape[0]),range(lev_wvfm[0].shape[1])):
                pos=sipm_abs_pos[(i,j)][0]
                if pos[0]==-1:
                    continue
                if sipm_rel_pos[(i,j)][0][0] ==(mod*2+itpc):
                    rect = mpatches.Rectangle((pos[2],-pos[1]), 1, 3,facecolor = cmap(c[i,j]))
                    ax[mod][itpc].add_patch(rect)
        elif light_mode == "sum_hit":
            for lhit in np.nditer(lev_hits):
                pos=det_bounds[(lhit["tpc"],lhit["det"])][0]
                if lhit["tpc"]==(mod*2+itpc):
                    intensity = norm(lhit["max"])
                    if not(np.isin(lhit["det"],acl_det)):
                           intensity *=3
                    rect = mpatches.Rectangle((pos[0,2],pos[0,1]), 1, 0.9*(pos[1,1]-pos[0,1]),facecolor ="red",alpha=intensity)
                    ax[mod][itpc].add_patch(rect)
        elif light_mode == "sipm_hit":
            for lhit in np.nditer(lev_hits):
                pos=sipm_abs_pos[(lhit["adc"],lhit["chan"])][0]
                if sipm_rel_pos[(lhit["adc"],lhit["chan"])][0][0] ==(mod*2+itpc):
                    rect = mpatches.Rectangle((pos[2],pos[1]), 1, 3,facecolor ="red",alpha=norm(lhit["max"]))
                    ax[mod][itpc].add_patch(rect)
                    
print("Spill: ",spill)                
spill += 1

In [ ]:
def data_readout(io_first, io_second, spill, event_id, h5file, one_entry_field, many_entry_field):

    fig = plt.figure(figsize=(14,8),tight_layout=True)
    subfigs = fig.subfigures(1, 6, wspace=0, width_ratios=[0.8,1.5,0.8,0.8,1.5,0.8], height_ratios=[1])
    axs0 = subfigs[0].subplots(8, 1,sharey=True,gridspec_kw={'hspace': 0})
    axs1 = subfigs[1].subplots(1, 1)
    axs2 = subfigs[2].subplots(8, 1,sharey=True,gridspec_kw={'hspace': 0})
    axs3 = subfigs[3].subplots(8, 1,sharey=True,gridspec_kw={'hspace': 0})
    axs4 = subfigs[4].subplots(1, 1)
    axs5 = subfigs[5].subplots(8, 1,sharey=True,gridspec_kw={'hspace': 0})

    cmap = cm.jet

    titles = ["mod. 2, io_group 3","mod. 1, io_group 1","mod. 2, io_group 4","mod. 1, io_group 2",
              "mod. 4, io_group 7","mod. 3, io_group 5","mod. 4, io_group 8","mod. 3, io_group 6"]
    colors = ['aqua','aqua','lightgreen','lightgreen','yellow','yellow','orangered','orangered']
    ios = [3,1,4,2,7,5,8,6]
    l_trap_type = [1,1,1,0,1,1,1,0]
    s_id_R = [18,19,20,4,21,22,23,5]
    s_id_L = [0,1,2,0,3,4,5,1]

    hits_trk = one_to_many(event_id, h5file, one_entry_field, many_entry_field)
    
    x_pos = hits_trk['x'] # [cm], drift direction
    io_groups = hits_trk['io_group']
    y_pos = hits_trk['y']
    z_pos = hits_trk['z'] 
    
    for i in range(len(io_groups)):
        if io_groups[i]==io_first:
            X = x_pos[i]
            Y = y_pos[i]
            Z = z_pos[i]
            axs1.scatter(Z,Y,c=colors[ios.index(io_first)],alpha=1,lw=1.5,s=10)
        elif io_groups[i]==io_second:
            X = x_pos[i]
            Y = y_pos[i]
            Z = z_pos[i]
            axs4.scatter(Z,Y,c=colors[ios.index(io_second)],alpha=1,lw=1.5,s=10)
        else:
            axs1.plot(0,0,c='navy',alpha=0.1)
            axs4.plot(0,0,c='navy',alpha=0.1)

    tpc_rectL = plt.Rectangle((-64.5,-65), 64, 130, linewidth=0.75, edgecolor='b', facecolor=cmap(0),zorder=-1)
    tpc_rectR = plt.Rectangle((0.5,-65), 64, 130, linewidth=0.75, edgecolor='b', facecolor=cmap(0),zorder=-1)

    axs1.add_patch(tpc_rectL)
    axs4.add_patch(tpc_rectR)
    axs1.set_aspect("equal")
    axs1.set_xlabel("z [cm]")
    axs1.set_ylim(-65,66)
    axs4.set_xlim(-65,0)
    axs1.set_title(titles[ios.index(io_first)])
    axs1.yaxis.set_ticklabels([]) 
    axs4.set_xlabel("z [cm]")
    axs4.set_ylim(-65,65)
    axs4.set_xlim(0,65)
    axs4.set_aspect("equal")
    axs4.set_title(titles[ios.index(io_second)])
    axs4.yaxis.set_ticklabels([]) 
    
    axs0[0].set_title("Left:\nio_group "+str(io_first))
    axs2[0].set_title("Right:\nio_group "+str(io_first))
    axs3[0].set_title("Left:\nio_group "+str(io_second))
    axs5[0].set_title("Right:\nio_group "+str(io_second))
    axs0[7].set_xlabel(r"Samples [0.01 $\mu$s]")
    axs2[7].set_xlabel(r"Samples [0.01 $\mu$s]")
    axs3[7].set_xlabel(r"Samples [0.01 $\mu$s]")
    axs5[7].set_xlabel(r"Samples [0.01 $\mu$s]")

    samples = f['light/wvfm/data']['samples']
    if (io_second%2)==0:    
        wvfm_evt_acl_L = samples[spill,io_first-2,0:32,:][valid_wvfm[0,0,0:32]==1]
        sum_ch_acl_L = wvfm_evt_acl_L.reshape(4, 6, 1000).sum(axis=1)
        wvfm_evt_lcm_L = samples[spill,io_first-1,0:32,:][valid_wvfm[0,1,0:32]==1]
        sum_ch_lcm_L = wvfm_evt_lcm_L.reshape(12, 2, 1000).sum(axis=1)
    
        wvfm_evt_acl_R = samples[spill,io_second-2,0:32,:][valid_wvfm[0,0,0:32]==1]
        sum_ch_acl_R = wvfm_evt_acl_R.reshape(4, 6, 1000).sum(axis=1)
        wvfm_evt_lcm_R = samples[spill,io_second-1,0:32,:][valid_wvfm[0,1,0:32]==1]
        sum_ch_lcm_R = wvfm_evt_lcm_R.reshape(12, 2, 1000).sum(axis=1)
        
    else:
        wvfm_evt_acl_L = samples[spill,io_first-2,32:,:][valid_wvfm[0,0,32:]==1]
        sum_ch_acl_L = wvfm_evt_acl_L.reshape(4, 6, 1000).sum(axis=1)
        wvfm_evt_lcm_L = samples[spill,io_first-1,32:,:][valid_wvfm[0,1,32:]==1]
        sum_ch_lcm_L = wvfm_evt_lcm_L.reshape(12, 2, 1000).sum(axis=1)
    
        wvfm_evt_acl_R = samples[spill,io_second-2,32:,:][valid_wvfm[0,0,32:]==1]
        sum_ch_acl_R = wvfm_evt_acl_R.reshape(4, 6, 1000).sum(axis=1)
        wvfm_evt_lcm_R = samples[spill,io_second-1,32:,:][valid_wvfm[0,1,32:]==1]
        sum_ch_lcm_R = wvfm_evt_lcm_R.reshape(12, 2, 1000).sum(axis=1)
    
    all_sums=[]  
    for i in range(8):
        if l_trap_type[i]==1:
            clr = 'greenyellow'
            wvfm_scndL = sum_ch_lcm_R[0+i,:]
            wvfm_scndR = sum_ch_lcm_R[5+i,:]
            wvfm_frstL = sum_ch_lcm_L[0+i,:]
            wvfm_frstR = sum_ch_lcm_L[5+i,:]
        else:
            clr = 'lightgreen'   
            tb = i//7
            wvfm_scndL = sum_ch_acl_R[0+tb,:]
            wvfm_scndR = sum_ch_acl_R[2+tb,:]
            wvfm_frstL = sum_ch_acl_L[0+tb,:]
            wvfm_frstR = sum_ch_acl_L[2+tb,:]
        
        all_sums.extend(wvfm_scndL+wvfm_scndR+wvfm_frstL+wvfm_frstR)
        y_min = (min(all_sums)-500)
        y_max = (max(all_sums))
    
        axs0[i].plot(np.linspace(0,SAMPLES-1,SAMPLES),wvfm_frstL,color='k')
        axs0[i].set_facecolor(clr) 
        #axs0[i].set_box_aspect(1)
        axs0[i].label_outer()
        axs0[i].set_ylim(y_min,y_max)
    
        axs2[i].plot(np.linspace(0,SAMPLES-1,SAMPLES),wvfm_frstR,color='k')
        axs2[i].set_facecolor(clr)
        axs2[i].label_outer()
        #axs2[i].set_box_aspect(1)
        axs2[i].set_ylim(y_min,y_max)
        axs2[i].yaxis.set_ticklabels([])
    
        axs3[i].plot(np.linspace(0,SAMPLES-1,SAMPLES),wvfm_scndL,color='k')
        axs3[i].set_facecolor(clr)
        axs3[i].label_outer()
        #axs3[i].set_box_aspect(1)
        axs3[i].set_ylim(y_min,y_max)
        axs3[i].yaxis.set_ticklabels([])
    
        axs5[i].plot(np.linspace(0,SAMPLES-1,SAMPLES),wvfm_scndR,color='k')
        axs5[i].set_facecolor(clr)
        axs5[i].label_outer()
        #axs5[i].set_box_aspect(1)
        axs5[i].set_ylim(y_min,y_max)
        axs5[i].yaxis.set_ticklabels([])
        
    plt.show()

In [ ]:
data_readout(3, 1, 4, 3, f, 'charge/events', 'charge/calib_final_hits')

In [ ]:
data_readout(4, 2, 4, 3, f, 'charge/events', 'charge/calib_final_hits')

In [ ]:
data_readout(7, 5, 4, 3, f, 'charge/events', 'charge/calib_final_hits')

In [ ]:
data_readout(8, 6, 4, 3, f, 'charge/events', 'charge/calib_final_hits')